# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [2]:
# import prerequisite python modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, log_loss, roc_auc_score

In [3]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv(r'C:\Users\ibast\DSI_production\05_src\adult\adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [4]:
adult_dt.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming you have already created the adult_dt DataFrame as shown previously

# Step 1: Create the feature (X) and target (Y) dataframes
X = adult_dt.drop(columns=['income'])  # Drop the target column
Y = adult_dt['income']  # Select the target column

# Step 2: Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Display the shapes of the resulting datasets to confirm
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")


X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The random_state parameter in the train_test_split function (and other scikit-learn functions) sets a seed for the random number generator used to split data into training and test sets. This parameter is essential for reproducibility, as it ensures the random processes, like shuffling and splitting, yield the same results each time the code runs with the same data.

Why is random_state useful?
Reproducibility: Setting random_state allows other researchers or developers to reproduce the exact splits or random processes in your experiments. Reproducibility is essential in scientific research and data science, as it ensures that others can verify, validate, and build upon your work under identical conditions.

Model Comparison: When testing or comparing different models, it’s essential to use the same training and test splits to ensure that performance differences are due to the models themselves, not variations in the data split. A fixed random_state guarantees consistent data splits for fair comparison.

Debugging: Consistent splits make it easier to debug or fine-tune model training, as variations due to data splits are eliminated, allowing you to focus on other variables in the modeling process.



*(Comment here.)*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

X = adult_dt.drop('income', axis=1)
y = adult_dt['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numerical_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

preprocessor = ColumnTransformer(
    transformers=[
    
        ('num', 
         Pipeline(steps=[
             ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')), 
             ('robust_scaler', RobustScaler())  
         ]), numerical_cols),
        
        # Categorical pipeline
        ('cat', 
         Pipeline(steps=[
             ('simple_imputer', SimpleImputer(strategy='most_frequent')),  # Mode Imputation
             ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))  # One-hot Encoding
         ]), categorical_cols)
    ]
)

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)


ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
encoded_feature_names = ohe.get_feature_names_out(categorical_cols)

feature_names = numerical_cols + list(encoded_feature_names)

X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=feature_names, index=X_train.index)
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=feature_names, index=X_test.index)

# Display the transformed DataFrames
print(X_train_transformed_df.head())
print(X_test_transformed_df.head())

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [9]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X = adult_dt.drop('income', axis=1)
y = adult_dt['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# numerical and categorical columns
numerical_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

preprocessor = ColumnTransformer(
    transformers=[
        # Numerical pipeline
        ('num', 
         Pipeline(steps=[
             ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),  
             ('robust_scaler', RobustScaler()) 
         ]), numerical_cols),
        
        ('cat', 
         Pipeline(steps=[
             ('simple_imputer', SimpleImputer(strategy='most_frequent')), 
             ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))  
         ]), categorical_cols)
    ]
)

# model pipeline
model_pipeline = Pipeline([
    ('preprocessing', preprocessor),             
    ('classifier', RandomForestClassifier())       
])

model_pipeline.fit(X_train, y_train)

# Predict on the test data
y_pred = model_pipeline.predict(X_test)

print(y_pred[:10]) 


[0 0 1 0 0 1 1 0 0 1]


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [11]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

cv_results = cross_validate(
    model_pipeline,      
    X_train,            
    y_train,            
    cv=5,                
    scoring=scoring,    
    return_train_score=True  
)
for metric in scoring.keys():
    print(f"\n{metric.upper()} - Training and Validation Results:")
    print(f"Training {metric}: {cv_results['train_' + metric]}")
    print(f"Validation {metric}: {cv_results['test_' + metric]}")
    print(f"Average Training {metric}: {cv_results['train_' + metric].mean():.4f}")
    print(f"Average Validation {metric}: {cv_results['test_' + metric].mean():.4f}")

c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(



NEG_LOG_LOSS - Training and Validation Results:
Training neg_log_loss: [-0.08070598 -0.08195462 -0.08144901 -0.08198176 -0.08195931]
Validation neg_log_loss: [-0.39445382 -0.35021271 -0.39649306 -0.36466    -0.38524183]
Average Training neg_log_loss: -0.0816
Average Validation neg_log_loss: -0.3782

ROC_AUC - Training and Validation Results:
Training roc_auc: [1. 1. 1. 1. 1.]
Validation roc_auc: [0.89509928 0.90506718 0.90207796 0.90581613 0.9047243 ]
Average Training roc_auc: 1.0000
Average Validation roc_auc: 0.9026

ACCURACY - Training and Validation Results:
Training accuracy: [1.         0.99995201 1.         1.         0.99990403]
Validation accuracy: [0.84702495 0.84971209 0.85355086 0.85563448 0.86004991]
Average Training accuracy: 1.0000
Average Validation accuracy: 0.8532

BALANCED_ACCURACY - Training and Validation Results:
Training balanced_accuracy: [1.         0.99990032 1.         1.         0.99980064]
Validation balanced_accuracy: [0.76364471 0.77385652 0.77012065 0.7

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

Calculate the mean of each metric. 

In [12]:
import pandas as pd

cv_results_df = pd.DataFrame(cv_results)

cv_results_df_sorted = cv_results_df.sort_values(by='test_neg_log_loss', ascending=False)

mean_results = cv_results_df.mean()
print("Fold-level Results Sorted by Negative Log Loss:")
print(cv_results_df_sorted)

print("\nMean Results Across All Folds:")
print(mean_results)


Fold-level Results Sorted by Negative Log Loss:
    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
1  14.667752    0.277882          -0.350213           -0.081955      0.905067   
3  15.665269    0.288437          -0.364660           -0.081982      0.905816   
4  16.953823    0.299644          -0.385242           -0.081959      0.904724   
0  18.869560    0.292064          -0.394454           -0.080706      0.895099   
2  14.447720    0.305879          -0.396493           -0.081449      0.902078   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
1            1.0       0.849712        0.999952                0.773857   
3            1.0       0.855634        1.000000                0.775587   
4            1.0       0.860050        0.999904                0.782852   
0            1.0       0.847025        1.000000                0.763645   
2            1.0       0.853551        1.000000                0.770121   

   train_balan

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.